In this tutorial, we are going to explore how different binary evolution assumptions are imprinted into observable features in the population of double white dwarfs that form. The techniques that we use here are applicable to other stellar remnants, but it's important to always carefully analyze the output of your simulations since physics can do a lot of counterintuitive things to binary star populations! It's also a good idea to keep the COSMIC documentation handy: https://cosmic-popsynth.github.io/COSMIC/


First, we need to install the code itself. In Google Colab, you can do this using the Python Package Interface (or PyPI), which is called through `pip`. You only need to do this one time, and once the install is complete, you'll be prompted to restart the notebook.

In [ ]:
!pip install cosmic-popsynth

First, we need to import two modules from cosmic: one that holds the initial states of the binaries (`InitialBinaryTable`) and the `Evolve` module which will evolve the initial binaries. We'll also import matplotlib so that we can make some plots.


In [ ]:
from cosmic.sample.initialbinarytable import InitialBinaryTable
from cosmic.evolve import Evolve

import matplotlib.pyplot as plt

Next, we want to sample an initial population of binaries that is drawn from different observationally motivated, but independent distribution functions. You can see a description of these choices in the ["sampling initial populations"](https://cosmic-popsynth.github.io/COSMIC/pages/runpop.html) portion of the documentation.

When you run the cell below, you should notice a warning. These are okay! In general, if you see a warning that comes out of a software package, it doesn't mean that something is *necessarily* wrong. It does mean that the developer wants you to know something, since there might be things going on in the background. In this case, cosmic requires that you specify an assumption for the binary fraction, but the orbital period disribution from [Moe+2019](https://ui.adsabs.harvard.edu/abs/2019ApJ...875...61M/abstract) also describes the binary fraction. This means that the `binfrac_model` choice is being overwritten.


In [ ]:
final_kstar1 = [10, 11, 12]
final_kstar2 = [10, 11, 12]

InitialBinaries, mass_singles, mass_binaries, n_singles, n_binaries = InitialBinaryTable.sampler(
    'independent', final_kstar1, final_kstar2, binfrac_model=0.5, primary_model='kroupa01',
    ecc_model='uniform', porb_model='moe19', m2_min=0.08, SF_start=13700.0,
    SF_duration=0.0, met=0.02, size=10000)


Now that we have a population of initial binaries, it's time to evolve them! Before we can do that, we need to make about 60 choices for what the physics should be doing in our binaries. That's a lot of choices, we know. In the cosmic documentation, we have a description of where each of these choices come from, with many choices being made based on different papers. You can peruse those choices, and create your own BSEDict or Params.ini file here: [cosmic-popsynth.github.io/COSMIC/pages/inifile.html](https://cosmic-popsynth.github.io/COSMIC/pages/inifile.html)


Once the BSEDict is specified, we'll use `Evolve.evolve` to (you guessed it) *evolve* our binary population. Since there are about 10,000 binaries, you're going to have to wait for the lovely Colab processors to return those binaries. That means we have about 3 minutes to answer any question you have about the process so far!


In [ ]:
### This cell will take like 3 minutes to run ###

BSEDict = {'xi': 1.0, 'bhflag': 1, 'neta': 0.5, 'windflag': 3, 'wdflag': 1, 'alpha1': 1.0,
           'pts1': 0.001, 'pts3': 0.02, 'pts2': 0.01, 'epsnov': 0.001, 'hewind': 0.5, 'ck': 1000,
           'bwind': 0.0, 'lambdaf': 0.0, 'mxns': 3.0, 'beta': -1.0, 'tflag': 1, 'acc2': 1.5,
           'grflag' : 1, 'remnantflag': 4, 'ceflag': 0, 'eddfac': 1.0, 'ifflag': 0,
           'bconst': 3000, 'sigma': 265.0, 'gamma': -2.0, 'pisn': 45.0,
           'natal_kick_array' : [[-100.0,-100.0,-100.0,-100.0,0.0], [-100.0,-100.0,-100.0,-100.0,0.0]],
           'bhsigmafrac' : 1.0, 'polar_kick_angle' : 90,
           'qcrit_array' : [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
           'cekickflag' : 2, 'cehestarflag' : 0, 'cemergeflag' : 0, 'ecsn' : 2.25, 'ecsn_mlow' : 1.6,
           'aic' : 1, 'ussn' : 0, 'sigmadiv' :-20.0, 'qcflag' : 1, 'eddlimflag' : 0,
           'fprimc_array' : [2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,
                             2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0],
           'bhspinflag' : 0, 'bhspinmag' : 0.0, 'rejuv_fac' : 1.0, 'rejuvflag' : 0, 'htpmb' : 1,
           'ST_cr' : 1, 'ST_tide' : 1, 'bdecayfac' : 1, 'rembar_massloss' : 0.5, 'kickflag' : 1,
           'zsun' : 0.019, 'bhms_coll_flag' : 0, 'don_lim' : -1, 'acc_lim' : -1, 'rtmsflag' : 0,
           'wd_mass_lim' : 1}

bpp, bcm, initC, kick_info = Evolve.evolve(initialbinarytable=InitialBinaries,
                                           BSEDict=BSEDict)




Once the evolution is complete, cosmic returns four DataFrames that have a common index for each binary that is stored in the `bin_num` column. Since we are working with double white dwarfs, we are going to ignore `kick_info` which holds kinematic information about natal kicks imparted to compact objects. All of the important events in the evolution of each binary are logged in the `bpp`

Most of the analysis that you can do with cosmic is best done using `pandas` a python module that is great for working with big tables of data. In the cell below, we can select the population of binaries that make double white dwarfs by selecting on the integer star type and making sure that there is a non-zero orbital period (which would indicate a merger or disruption). You can see what the different star types translate to here in the documentation: [cosmic-popsynth.github.io/COSMIC/pages/output_info.html](https://cosmic-popsynth.github.io/COSMIC/pages/output_info.html).


In [ ]:
bpp_DWD = bpp.loc[(bpp.kstar_1.isin([10,11,12])) & (bpp.kstar_2.isin([10,11,12])) & (bpp.porb > 0)].groupby('bin_num', as_index=False).first()

In the cell below, we can make a histogram of the orbital period, noting that cosmic uses days for the units of the orbital period. What do you notice about the distribution of the orbital periods?

In [ ]:
plt.hist(bpp_DWD.porb)
plt.xlabel('orbital period [days]')

In the cell below, let's modify the histogram to better show the range of the data.

Now that we have an updated figure, what do you notice about the orbital period distribution?

The shape of the two bumps is driven by mass transfer interactions between the double white dwarf progenitor stars. Most of the binaries go through common envelope -- we can count how many binaries experienced a common envelope using the `evol_type` column in the `bpp`. My favorite way to do this is to find bin_nums for binaries that go through a common envelope, then combine those with the DWD bin_nums.

In [ ]:
CE_bin_num = bpp.loc[bpp.evol_type == 7].bin_num.unique()

In [ ]:
DWD_CE = bpp_DWD.loc[bpp_DWD.bin_num.isin(CE_bin_num)]

In [ ]:
DWD_CE

In the cell below make a histogram of the orbital periods for all DWDs and just the DWDs which went through a CE. What do you see? How would we change the shape of the period distribution?


Let's make a new BSEDict that changes the α parameter for common envelepe (this is saved as alpha1 in the BSEDict). One thing that's cool about cosmic is that you can rerun the same binaries, but with different model assumptions to see how those choices affect the binary population.

In the cell below, find `alpha1` and modify to a value that is larger than zero. Based on the value you supplied, and what you know about common envelope, what do you think will happen to your orbital period distribution?

In [ ]:
BSEDict_alpha_change = {'xi': 1.0, 'bhflag': 1, 'neta': 0.5, 'windflag': 3, 'wdflag': 1, 'alpha1': 1.0,
           'pts1': 0.001, 'pts3': 0.02, 'pts2': 0.01, 'epsnov': 0.001, 'hewind': 0.5, 'ck': 1000,
           'bwind': 0.0, 'lambdaf': 0.0, 'mxns': 3.0, 'beta': -1.0, 'tflag': 1, 'acc2': 1.5,
           'grflag' : 1, 'remnantflag': 4, 'ceflag': 0, 'eddfac': 1.0, 'ifflag': 0,
           'bconst': 3000, 'sigma': 265.0, 'gamma': -2.0, 'pisn': 45.0,
           'natal_kick_array' : [[-100.0,-100.0,-100.0,-100.0,0.0], [-100.0,-100.0,-100.0,-100.0,0.0]],
           'bhsigmafrac' : 1.0, 'polar_kick_angle' : 90,
           'qcrit_array' : [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],
           'cekickflag' : 2, 'cehestarflag' : 0, 'cemergeflag' : 0, 'ecsn' : 2.25, 'ecsn_mlow' : 1.6,
           'aic' : 1, 'ussn' : 0, 'sigmadiv' :-20.0, 'qcflag' : 1, 'eddlimflag' : 0,
           'fprimc_array' : [2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,
                             2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0,2.0/21.0],
           'bhspinflag' : 0, 'bhspinmag' : 0.0, 'rejuv_fac' : 1.0, 'rejuvflag' : 0, 'htpmb' : 1,
           'ST_cr' : 1, 'ST_tide' : 1, 'bdecayfac' : 1, 'rembar_massloss' : 0.5, 'kickflag' : 1,
           'zsun' : 0.019, 'bhms_coll_flag' : 0, 'don_lim' : -1, 'acc_lim' : -1, 'rtmsflag' : 0,
           'wd_mass_lim' : 1}

Next, we will rerun the evolution call, but be sure to save into a new set of DataFrames.

In [ ]:
bpp_alpha_change, bcm_alpha_change, initC_alpha_change, kick_info_alpha_change = Evolve.evolve(initialbinarytable=InitialBinaries,
                                           BSEDict=BSEDict_alpha_change)

In the cells below, make a histogram that selects the DWDs and compares the orbital period distributions of the two different populations. Do they do what you expect? If you want to go the extra distance, peruse the cosmic docs and find some other parameters to change.